In [27]:
import re
import matplotlib.pyplot as plt
import os
from sklearn.feature_extraction.text import CountVectorizer
#from sklearn import cross_validation
from sklearn import model_selection
import os

import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn import linear_model, datasets

#
def load_one_flle(filename):
    x=[]
    with open(filename) as f:
        line=f.readline()
        line=line.strip('\n')
    return line

#加载ADFA-LD中的正常样本数据
def load_adfa_training_files(rootdir):
    x=[]
    y=[]
    list = os.listdir(rootdir)
    for i in range(0, len(list)):
        path = os.path.join(rootdir, list[i])
        if os.path.isfile(path):
            x.append(load_one_flle(path))
            #print("Load file(%s)" % path)
            y.append(0)
    return x,y

#定义遍历目录下文件的函数
def dirlist(path, allfile):
    filelist = os.listdir(path)

    for filename in filelist:
        filepath = os.path.join(path, filename)
        if os.path.isdir(filepath):
            dirlist(filepath, allfile)
        else:
            allfile.append(filepath)
    return allfile

#从攻击数据集中筛选和Java溢出攻击相关的数据
def load_adfa_java_files(rootdir):
    x=[]
    y=[]
    allfile=dirlist(rootdir,[])
   # print(allfile)
    for file in allfile:
        if re.match(r"ADFA-LD/Attack_Data_Master/Java_Meterpreter_\d+\\UAD-Java-Meterpreter*",file):
           # print("Load file(%s)" % file)
            x.append(load_one_flle(file))
            y.append(1)#增加标签
    return x,y



if __name__ == '__main__':
    #特征化

    x1,y1=load_adfa_training_files("ADFA-LD/Training_Data_Master/")
    x2,y2=load_adfa_java_files("ADFA-LD/Attack_Data_Master/")
    #print(x1)
    #print ("x2:",x2)

    x=x1+x2
    y=y1+y2
    #print x
    vectorizer = CountVectorizer(min_df=1)
    x=vectorizer.fit_transform(x)
    x=x.toarray()

    mlp = MLPClassifier(hidden_layer_sizes=(150,50), max_iter=10, alpha=1e-4,
                        solver='sgd', verbose=10, tol=1e-4, random_state=1,
                        learning_rate_init=.1)

    #训练样本
    logreg = linear_model.LogisticRegression(solver='liblinear',C=1e5,max_iter=3000)#实例化逻辑回归算法，正则系数为1e5
     #使用十折交叉验证
    score=model_selection.cross_val_score(logreg, x, y, n_jobs=1, cv=10)
    
    print(np.mean(score))

  

0.936137185747875


sklearn 中的cross_val_score函数可以用来进行交叉验证，因此十分常用，这里介绍这个函数的参数含义。

sklearn.cross_validation.cross_val_score(estimator, X, y=None, scoring=None,
cv=None, n_jobs=1, verbose=0, fit_params=None, pre_dispatch=‘2*n_jobs’)

estimator:估计方法对象(分类器)
X：数据特征(Features)
y：数据标签(Labels)
soring：调用方法(包括accuracy和mean_squared_error等等)
cv：几折交叉验证
n_jobs：同时工作的cpu个数（-1代表全部）